### Run a notebook in Azure using AzureML

This requires an AzureML workspace to be set up. The workspace will be loaded from the configuration using `Workspace.from_config()`. 

Please see [this notebook](https://github.com/Azure/MachineLearningNotebooks/blob/master/configuration.ipynb) on how to create an AzureML workspace.

In [118]:
import sys
sys.path.append("../../")

import azureml.core

print("This notebook was created using version 1.0.8 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

This notebook was created using version 1.0.8 of the Azure ML SDK
You are currently using version 1.0.15 of the Azure ML SDK


### Get the workspace and the experiment to run under (create a new one each day)

In [108]:
from azureml.core import Workspace
from azureml.core import Experiment
import datetime

ws = Workspace.from_config()
experiment = Experiment(ws, 'recommender-' + datetime.datetime.now().strftime("%Y-%m-%d"))

Found the config file in: /data/home/danielsc/notebooks/Recommenders/config.json


### Making sure the yaml file defining the environment is present

In [112]:
!(cd ../.. && {sys.executable} scripts/generate_conda_file.py)

Generated conda file: reco_base.yaml

To create the conda environment:
$ conda env create -f reco_base.yaml

To update the conda environment:
$ conda env update -f reco_base.yaml

To register the conda environment in Jupyter:
$ conda activate reco_base
$ python -m ipykernel install --user --name reco_base --display-name "Python (reco_base)"



### Setting up the job to run the notebook
For the environment we use the reco_base.yaml. Due to [this bug](https://msdata.visualstudio.com/Vienna/_workitems/edit/377078), the compute target needs to be DSVM right now, so you need to attach a DSVM to your workspace -- mine is called `"lidsvm"`.  

In [129]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.core.script_run_config import ScriptRunConfig

run_config = RunConfiguration()
run_config.target = "lidsvm"
run_config.environment.docker.enabled = True
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE
run_config.environment.python.user_managed_dependencies = False
run_config.auto_prepare_environment = True
run_config.environment.python.conda_dependencies = CondaDependencies(conda_dependencies_file_path='../../reco_base.yaml')

script_run_config = ScriptRunConfig(source_directory='../../',
                                    script='scripts/notebook_launcher.py',
                                    arguments=['--notebook_path', 'notebooks/00_quick_start/fastai_movielens.ipynb', 'MOVIELENS_DATA_SIZE', '100k'],
                                    run_config=run_config)

run = experiment.submit(script_run_config)

run

Experiment,Id,Type,Status,Details Page,Docs Page
recommender-2019-02-23,recommender-2019-02-23_1550966125_c6b27450,azureml.scriptrun,Running,Link to Azure Portal,Link to Documentation


In [130]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [128]:
run.get_file_names()

['azureml-logs/60_control_log.txt',
 'azureml-logs/80_driver_log.txt',
 'azureml-logs/azureml.log',
 'driver_log',
 'outputs/output.ipynb']

In [127]:
!cat ../../scripts/notebook_launcher.py

import sys
import os
import papermill as pm
import argparse
from azureml.core import Run

os.system(sys.executable + ' -m ipykernel install --user --name ipyazureml')

def execute_notebook(notebook_path, parameters):
    pm.execute_notebook(
        notebook_path,
        'outputs/output.ipynb',
        kernel_name='ipyazureml',
        parameters=parameters,
    )
    results = pm.read_notebook('outputs/output.ipynb').dataframe.set_index("name")["value"]

    run = Run.get_context()
    for key, value in results.items():
        run.log(key, value)
            
    return results


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--notebook_path')
    FLAGS, unparsed = parser.parse_known_args()

    parameters= {unparsed[i]: unparsed[i+1] for i in range(0, len(unparsed), 2)}
    print(FLAGS.notebook_path, parameters)
    execute_notebook(notebook_path = FLAGS.notebook_path, 
                       parameters=parameters)


